In [2]:
#first begin with imports
import warnings
warnings.filterwarnings("ignore")
import env
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pydataset import data
from statsmodels.formula.api import ols


from scipy import stats
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score

from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import f_regression, RFE
from sklearn.metrics import mean_squared_error
from math import sqrt
import matplotlib.pyplot as plt

from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [3]:
#load tips dataset and take a look
tips = data('tips')
tips

,total_bill,tip,sex,smoker,day,time,size
1,16.99,1.01,Female,No,Sun,Dinner,2
2,10.34,1.66,Male,No,Sun,Dinner,3
3,21.01,3.50,Male,No,Sun,Dinner,3
4,23.68,3.31,Male,No,Sun,Dinner,2
5,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
240,29.03,5.92,Male,No,Sat,Dinner,3
241,27.18,2.00,Female,Yes,Sat,Dinner,2
242,22.67,2.00,Male,Yes,Sat,Dinner,2
243,17.82,1.75,Male,No,Sat,Dinner,2


1a.) Create a column named tip_percentage. This should be the tip amount divided by the total bill.

In [4]:
#making a new column which is tip divided by total bill and rounded to 2 decimal places
tips['tip_percentage'] = round(tips['tip']/tips['total_bill'],2)

In [5]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size,tip_percentage
1,16.99,1.01,Female,No,Sun,Dinner,2,0.06
2,10.34,1.66,Male,No,Sun,Dinner,3,0.16
3,21.01,3.50,Male,No,Sun,Dinner,3,0.17
4,23.68,3.31,Male,No,Sun,Dinner,2,0.14
5,24.59,3.61,Female,No,Sun,Dinner,4,0.15


In [6]:
tips.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 244 entries, 1 to 244
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   total_bill      244 non-null    float64
 1   tip             244 non-null    float64
 2   sex             244 non-null    object 
 3   smoker          244 non-null    object 
 4   day             244 non-null    object 
 5   time            244 non-null    object 
 6   size            244 non-null    int64  
 7   tip_percentage  244 non-null    float64
dtypes: float64(3), int64(1), object(4)
memory usage: 17.2+ KB


1b.) Create a column named price_per_person. This should be the total bill divided by the party size.

In [7]:
#make sure to use brackets since size is a keyword
tips['price_per_person'] = (tips['total_bill'])/(tips['size'])

In [8]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size,tip_percentage,price_per_person
1,16.99,1.01,Female,No,Sun,Dinner,2,0.06,8.495000
2,10.34,1.66,Male,No,Sun,Dinner,3,0.16,3.446667
3,21.01,3.50,Male,No,Sun,Dinner,3,0.17,7.003333
4,23.68,3.31,Male,No,Sun,Dinner,2,0.14,11.840000
5,24.59,3.61,Female,No,Sun,Dinner,4,0.15,6.147500



1c.) Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount? The tip percentage?

I think the most important for tip amount is

size
time
I think the most important for tip percentage is

total bill
1d.) Use all the other numeric features to predict tip amount. Use select k best and recursive feature elimination to select the top 2 features. What are they?

In [9]:
target = 'tip'

In [10]:
X_train, y_train, X_validate, y_validate, X_test, y_test = fe.train_validate_test(tips, target)

NameError: name 'fe' is not defined